In [113]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
import math

In [114]:
df = pd.read_csv("../data/small.csv")

In [115]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,author,subreddit,score,text_length,user_id,channel_id
0,0,0,0,ThatRookieGuy80,AskMen,7.0,10,0,0
1,8,8,8,Gr000vy,backpain,5.0,13,1,1
2,20,20,20,623-252-2424,PublicFreakout,2.0,2,2,2
3,22,22,22,Soviet-Biscuit,mildlyinfuriating,2.0,5,3,3
4,44,44,44,judyclimbs,MadeMeSmile,8.0,37,4,4
...,...,...,...,...,...,...,...,...,...
357899,5088996,5908310,5089075,It_me_aiden,teenagersbutpog,2.0,3,6104,1399
357900,5089001,5908315,5089080,scarecrow9281,girlsfrontline,4.0,2,6239,4164
357901,5089003,5908317,5089082,trimaximusrt,AskReddit,17.0,2,2583,47
357902,5089017,5908331,5089096,hosehliao,singapore,2.0,5,4170,640


In [116]:
N = 10000
M = 10000

In [117]:
df.shape

(357904, 9)

In [118]:
user_to_channel = {}
channel_to_user = {}
to_rating = {}

In [119]:
A = df.score.apply(lambda x: math.log10(abs(x)+2))
df.loc[:,"norm_score"] = preprocessing.normalize([A],axis=1)[0]
B = df.text_length.apply(lambda x: math.log10(abs(x)+1))
df.loc[:,"norm_length"] = preprocessing.normalize([B],axis=1)[0]

In [120]:
df.score.mean()
df.score.std()
norm_score = (df.score-df.score.mean())/df.score.std()

In [121]:
df.loc[:, "norm_score"] = norm_score

In [122]:
df.text_length.mean()
df.text_length.std()
norm_length = (df.text_length-df.text_length.mean())/df.text_length.std()
norm_length = (norm_length-norm_length.mean())/norm_length.std()

In [123]:
df.loc[:, "norm_length"] = norm_length

In [124]:
from sklearn.decomposition import PCA
A = pd.DataFrame({"norm_score": norm_score, "norm_length": norm_length})
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(A)
df.loc[:, "PC"] = principalComponents


In [125]:
df.drop(columns = ["Unnamed: 0.2", "Unnamed: 0.1", "Unnamed: 0", "score", "text_length"], inplace = True)

In [126]:
df = df.groupby(["user_id","channel_id"]).sum().reset_index()

/var/folders/k_/cq1sx9d94zs5s6d39hnbqppm0000gn/T/ipykernel_1839/2513613624.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(["user_id","channel_id"]).sum().reset_index()


In [127]:
df = shuffle(df)
cut = int(0.6*len(df))
cut2 = int(0.8 * len(df))
val = df[cut:cut2]
train = df[0:cut]
test = df[cut2:]

In [128]:
count = 0
def update_transform(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Already processed: ", float(count)/float(cut))
    if row.norm_score + row.norm_length <= 100: 
        i = int(row.user_id)
        j = int(row.channel_id)
        
        if i not in user_to_channel:
            user_to_channel[i] = [j]
        else:
            user_to_channel[i].append(j)
        
        if j not in channel_to_user:
            channel_to_user[j] = [i]
        else:
            channel_to_user[j].append(i)
        
        to_rating[(i,j)] = [row.norm_score]
train.apply(update_transform, axis = 1)

85946    None
82162    None
29876    None
64988    None
26970    None
         ... 
61919    None
3571     None
21959    None
5822     None
8684     None
Length: 52624, dtype: object

In [129]:
count = 0
to_rating_test = {}
def update_transform_2(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Already processed: ", float(count)/float(cut))
    if row.norm_score + row.norm_length <= 100:
        i = int(row.user_id)
        j = int(row.channel_id)
        to_rating_test[(i,j)] = [row.norm_score]
test.apply(update_transform_2, axis = 1)
to_rating_val = {}
cuont = 0
def update_transform_3(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("Already processed: ", float(count)/float(cut))
    if row.norm_score + row.norm_length <= 100:
        i = int(row.user_id)
        j = int(row.channel_id)
        to_rating_val[(i,j)] = [row.norm_score]
val.apply(update_transform_3, axis = 1)

55961    None
80575    None
76229    None
59895    None
58023    None
         ... 
45793    None
36518    None
7878     None
12195    None
18098    None
Length: 17541, dtype: object

In [130]:
with open('../data/user_to_channel.json', 'wb') as f:
    pickle.dump(user_to_channel, f)
with open('../data/channel_to_user.json', 'wb') as f:
    pickle.dump(channel_to_user, f)
with open('../data/to_rating.json', 'wb') as f:
    pickle.dump(to_rating, f)
with open('../data/to_rating_test.json', 'wb') as f:
    pickle.dump(to_rating_test, f)
with open('../data/to_rating_val.json', 'wb') as f:
    pickle.dump(to_rating_val, f)